# 检查所见添加年龄性别并分组

### 年龄划分标准:3-6岁;7-12岁;13-17岁;18-45岁;46-69岁;>69岁

In [9]:
from tqdm import tqdm
import pandas as pd 
import numpy as np

new_finding =[]

csv = pd.read_csv('f2i_with_age_clean.csv', encoding='utf-8', usecols=['clear_finding', 'impression', 'age', 'sex'])

print(len(csv))

for index, row in tqdm(csv.iterrows()):
    age = int(row['age'])
    if age >= 3:    
        if age <=6:
            age_type = '幼儿'
        elif age <=12:
            age_type = '儿童'
        elif age <= 17:
            age_type = '少年'
        elif age <= 45:
            age_type = '青年'
        elif age <= 69: 
            age_type = '中年'
        elif age <= 100:
            age_type = '老年'
    else:
        new_finding.append(np.nan)  # 清洗不合格年龄数据
        continue
    new_finding.append(age_type + '。' + row['sex'] + '。' + row['clear_finding'])

    
csv['clear_finding'] = new_finding
csv = csv.dropna(axis='index', how='any').reset_index(drop=True)
csv = csv[['age', 'sex','clear_finding', 'impression']]

target_path = 'f2i_with_age.csv'
csv.to_csv(target_path, encoding='utf-8', index=None)


FileNotFoundError: [Errno 2] No such file or directory: 'clear_f2i_8_8.csv'

In [27]:
import re
def clear_impression(impression):
        sent_cleaner = lambda t: re.sub(
            '；+',  # 清洗多个连续分号
            '；',
            re.sub('\d(?:、|\.|：|:|;|，)(?=\D)',  # 匹配标题符，匹配(1、 1: 1.)后向界定为非数字模式
                   '；',
                   re.sub(
                       '[。?？;!！（）()]',  # 匹配非法字符
                       '；',
                       t.lower().strip().replace('"', '').replace('/', '、').replace('\\', '').replace("'", '').
                           replace("\r\n", '；').replace(' ', '').replace(',', '，').replace('\n', '；').replace('\r', '').replace(' ', '').
                           replace('————', '').replace('--', '-').replace('~', '-')
                   )))
        impression = sent_cleaner(impression)
        # 清洗无用日期
        res = re.search('[0-9]{4}-\d+-\d+', impression)
        if res:
            impression = impression.replace(res.group(), '前')
            
        # 清洗 3-6 为 3、4、5、6
#         res = (re.search('\d+-\d+', impression))
#         if res:
#             target = res.group()
#             impression = impression.replace(target, '、'.join([str(i) for i in range(int(target[:target.index('-')]), int(target[target.index('-')+1:])+1)]))
#         impression = re.sub('；+', '；',impression)
        
        # Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
        impression = impression.strip('、').strip('，').strip('；').strip('.')
        impression += '。'
        
        return impression

# 清洗检查印象

In [28]:
import pandas as pd
csv = pd.read_csv('f2i_with_age.csv')
new_impression = [clear_impression(row['impression']) for index, row in tqdm(csv.iterrows())]
csv['impression'] = new_impression

csv.to_csv('f2i_with_age_clean.csv', index=False)

112905it [00:03, 35870.63it/s]


## 制作每个类别和对应印象字典的数据

In [30]:
import warnings
from collections import Counter
warnings.filterwarnings("ignore")  # 忽略 append 警告
import time
start_time = time.time()

csv = pd.read_csv('f2i_with_age_clean.csv')
index = 0
temp = []

clear_finding_list = list(csv.to_dict()['clear_finding'].values())  
freq_dict = pd.Series(clear_finding_list).value_counts().to_dict()
freq_list = list(freq_dict.keys())  # 类别名字
count_list = list(freq_dict.values())  # 对应的类别个数

group = csv.groupby('clear_finding')  # 数据分组


for i in freq_list:  
    x = dict(Counter(list(group.get_group(i)['impression'])))  # 种类 i 对应的检查印象频率统计
    y2 = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}  # sort
    temp.append([i, count_list[index], y2])
    index += 1

df_total = pd.DataFrame(temp, columns=['clear_finding', 'frequency', 'impression'])

end_time = time.time()
print(f'total time: {(end_time - start_time):.1f}s.')
df_total.to_csv('dic_with_age_clean.csv', index = None)



total time: 3.6s.


In [13]:
df_total[-100:]

,clear_finding,frequency,impression
32383,中年。男。气管欠居中；右肺中野、右肺下野肺纹理增多；右肺上野见结节状高密度影。,1,{'右上肺结核（浸润型）。': 1}
32384,老年。男。左肺下野见斑片状高密度影，右肺中野见索条状高密度影。,1,{'右中肺炎性变，左下肺感染，建议治疗后复查。': 1}
32385,中年。女。两肺肺纹理增多、增粗、模糊；右侧部分肋骨骨质不连续。,1,{'请结合临床': 1}
32386,中年。男。右肺下野内带肺纹理增粗、紊乱、模糊。,1,{'右肺纹理增粗。': 1}
32387,老年。男。胸廓呈桶状、肋间隙增宽；两肺肺纹理增多、增粗、紊乱；肺野透亮度增高；两侧膈面低平；...,1,{'1、慢性支气管炎2、双肺气肿': 1}
...,...,...,...
32478,中年。男。两肺肺纹理增多、增粗；右肺上野中带见结节状高密度影，右肺中野中带见索条状高密度影。,1,{'左中肺野结节影伴纤维化灶，建议CT检查。': 1}
32479,老年。男。肋间隙增宽；两肺肺纹理增多、增粗、紊乱、模糊；左肺中野、左肺下野、右肺中野、右肺下...,1,{'胸部所示符合慢性支气管炎、肺气肿； 两肺结节考虑； 请结合临床或进检。': 1}
32480,中年。女。两肺肺纹理增多、紊乱；左肺下野中带见结节状高密度影。,1,{'左下肺野可疑结节影，乳头投影待除外，请随访复查。': 1}
32481,老年。女。两肺肺纹理增多、紊乱；两侧肺门形态欠规整；主动脉迂曲，主动脉增宽，主动脉球部钙化；...,1,{'1.两肺纹理增多紊乱。 2.主动脉粥样硬化。 3.片内诸骨骨质疏松改变。 请结合临...


## 制作只含肋骨的统计字典

In [ ]:
import warnings
from collections import Counter
warnings.filterwarnings("ignore")  # 忽略 append 警告
import time
start_time = time.time()

temp = []

for index, i in enumerate(freq_list):
    if '肋骨' in i:
        x = dict(Counter(list(group.get_group(i)['impression'])))  # 种类 i 对应的检查印象频率统计
        y2 = {k: v for k, v in sorted(x.items(), key=lambda item: item[1], reverse=True)}  # sort
        temp.append([i, count_list[index], y2])

        
df_rib = pd.DataFrame(temp, columns=['clear_finding', 'frequency', 'impression'])

end_time = time.time()
print(f'total time: {(end_time - start_time):.1f}s.')

In [ ]:
df_rib.to_csv('f2i_rib_with_age.csv', index=False)
df_rib[:4]

# 生成新的训练数据

In [14]:
from tqdm import tqdm
import random
import pandas as pd 

data = pd.read_csv('dic_with_age.csv')

fre = data['frequency']
frequency_list = fre.to_list()

# 所有数据
temp = []
for index, row in tqdm(data.iterrows()):    
    if frequency_list[index] > 50:
        for i in range(50):
            temp.append([row['clear_finding'], frequency_list[index], random.choice(list(eval(row['impression']).keys()))])
    elif frequency_list[index] < 20:
        for i in range(20 - frequency_list[index]):
            temp.append([row['clear_finding'], frequency_list[index], random.choice(list(eval(row['impression']).keys()))])
    else:
        for k in list(eval(row['impression']).keys()):
            temp.append([row['clear_finding'], frequency_list[index], k])
    
csv = pd.DataFrame(temp, columns=['clear_finding', 'frequency','impression'])

32483it [00:13, 2430.30it/s]


In [21]:
new_impression = [clear_impression(row['impression']) for index, row in tqdm(csv.iterrows())]

602738it [00:17, 35059.47it/s]


In [22]:
csv['impression'] = new_impression

In [25]:
csv.to_csv('new_train_data.csv', index=False)

## 检查印象数据清洗函数

In [16]:
def clear_impression(impression):
        sent_cleaner = lambda t: re.sub(
            '；+',  # 清洗多个连续分号
            '；',
            re.sub('\d(?:、|\.|：|:|;|，)(?=\D)',  # 匹配标题符，匹配(1、 1: 1.)后向界定为非数字模式
                   '；',
                   re.sub(
                       '[。?？;!！（）()]',  # 匹配非法字符
                       '；',
                       t.lower().strip().replace('"', '').replace('/', '、').replace('\\', '').replace("'", '').
                           replace("\r\n", '；').replace(' ', '').replace(',', '，').replace('\n', '；').replace('\r', '').replace(' ', '').
                           replace('————', '').replace('--', '-').replace('~', '-')
                   )))
        impression = sent_cleaner(impression)
        # 清洗无用日期
        res = re.search('[0-9]{4}-\d+-\d+', impression)
        if res:
            impression = impression.replace(res.group(), '前')
            
        # 清洗 3-6 为 3、4、5、6
#         res = (re.search('\d+-\d+', impression))
#         if res:
#             target = res.group()
#             impression = impression.replace(target, '、'.join([str(i) for i in range(int(target[:target.index('-')]), int(target[target.index('-')+1:])+1)]))
#         impression = re.sub('；+', '；',impression)
        
        # Python strip() 方法用于移除字符串头尾指定的字符（默认为空格或换行符）或字符序列。
        impression = impression.strip('、').strip('，').strip('；').strip('.')
        impression += '。'
        # print(repr(empression)) # 测试输出
        # time.sleep(0.2)
    
        return impression

In [11]:
import re
re.sub("、*['左'|'右']侧锁骨、*", '', '老年。男。左侧肺门增浓；两侧肋膈角变钝；右侧锁骨、右侧部分肋骨形态欠规整。')


'老年。男。左侧肺门增浓；两侧肋膈角变钝；右侧部分肋骨形态欠规整。'

In [18]:
def get_rib(text):

            res = re.split("[。|；]", text)  # 分句

            ret = []
            for i in res:
                if '肋骨' in i:
                    sub_res = re.split("，", i)
                    if len(sub_res) >= 2:
                        rib = '，'.join([i for i in sub_res if '锁骨' not in i])

                    elif len(sub_res) == 1:
                        rib = re.sub("、*['左'|'右']侧锁骨、*", '', sub_res[0])

                    rib = rib.replace('骨质不连续', '骨折').replace('形态欠规整', '陈旧性骨折').replace('见', '')
                    ret.append(rib)
            return ret

In [23]:
get_rib('老年。男。胸廓欠对称；两侧肺门增浓；心影呈横位；主动脉迂曲，主动脉增宽，主动脉球部钙化；两侧膈面局部膨隆；右侧部分肋骨、右侧锁骨形态欠规整。')

['右侧部分肋骨陈旧性骨折']

In [17]:
import re
# test = '左侧第8-12肋骨骨折。'
test = '1.两肺未见明显实质性病变。2.左胸3--5肋骨示有骨折征象。结合临床。3.主动脉硬化。'
t = clear_impression(test)
t

'两肺未见明显实质性病变；左胸3-5肋骨示有骨折征象；结合临床；主动脉硬化。'

# 生成新的肋骨训练数据

In [265]:
import time

start_time = time.time()

temp = []
for i in range(len(data)):
    result = data.loc[[i]]
    impre_dict = [eval(j) for j in result['impression']][0]
    find_str = [x for x in result['clear_finding']][0]
    
    for j in list(impre_dict.keys()):
        if '肋骨' in j:            
            temp.append([find_str, j,clear_impression(j)])
            
            
end_time = time.time()

df = pd.DataFrame(temp, columns=['clear_finding', 'origin','impression'])
print(f'total time: {(end_time - start_time):.1f}s.')

5-6
6-7
8-9
9-10
5-6
5-7
6-7
6-8
6-8
6-8
4-7
4-7
4-7
5-7
5-7
5-7
6-8
7-9
4-6
4-6
6-8
6-8
4-6
4-6
4-6
11-12
9-11
7-9
8-10
5-8
9-11
3-5
3-5
4-7
5-7
5-7
8-9
8-10
8-10
3-4
3-6
4-5
6-8
6-8
4-5
5-7
8-10
2-5
3-5
3-5
4-7
11-12
4-8
7-10
8-11
8-11
9-10
5-6
2-4
3-6
6-9
3-7
5-9
5-9
3-6
4-10
5-6
3-5
3-6
9-11
9-11
8-10
7-11
3-6
6-9
5-7
5-9
3-5
9-11
4-8
3-7
4-7
3-6
6-9
5-9
3-6
6-9
2-7
4-6
4-7
5-6
2-7
5-7
3-6
5-7
4-7
2-7
6-8
3-7
6-10
3-7
8-9
6-7
7-8
3-5
7-9
3-8
3-7
2-9
6-8
5-7
6-8
7-10
6-8
5-8
2-4
3-10
3-7
4-5
3-5
5-8
3-5
3-9
7-9
3-10
1-3
8-10
2-10
4-6
9-11
3-6
5-7
4-6
3-5
2-4
3-6
4-6
5-7
5-6
2-6
3-5
10-12
9-10
3-4
3-4
4-9
5-7
4-5
1-5
7-8
5-6
4-5
6-8
6-8
9-11
5-9
4-10
2-7
3-7
6-8
9-11
6-10
6-10
2-4
3-9
3-5
3-7
8-9
total time: 0.8s.


In [ ]:
find_str

In [272]:
test = df[-50:]
test.to_csv('50_rib_data.csv', index=False)
test

,clear_finding,origin,impression
3396,两肺肺纹理增多、增粗；左侧第七肋骨形态欠规整，左侧第六肋骨骨质不连续。,1、左侧第6肋骨骨质，第7肋骨疑似骨折，请结合临床，建议CT三维肋骨重建。2、左侧肩关节骨质...,左侧第6肋骨骨质，第7肋骨疑似骨折，请结合临床，建议ct三维肋骨重建；左侧肩关节骨质未见明显异常。
3397,两肺肺纹理增多、增粗、紊乱、模糊；左肺中野、左肺下野、右肺中野、右肺下野见大片状高密度影，境...,慢性支气管炎伴感染。\r\n右侧肋骨骨折，请结合临床。,慢性支气管炎伴感染；右侧肋骨骨折，请结合临床。
3398,两肺肺纹理增多、增粗；右侧第六、七后肋骨质不连续。,右侧第6、7肋骨骨折，请结合临床必要时CT检查。,右侧第6、7肋骨骨折，请结合临床必要时ct检查。
3399,两肺肺纹理增多、增粗、紊乱；两肺见斑片状高密度影，两肺见大片状高密度影，呈外高内低状；两侧胸...,1、两肺炎症/挫伤，随诊；两侧胸膜增厚；两侧胸腔积液；\r\n2、右侧多发肋骨骨折内固定术后...,两肺炎症、挫伤，随诊；两侧胸膜增厚；两侧胸腔积液；右侧多发肋骨骨折内固定术后改变；两侧部分肋...
3400,两肺肺纹理增多、增粗；左侧胸膜增厚；左侧第五、六、七、八肋骨形态欠规整。,左侧第5、6、7、8肋骨陈旧性骨折，请结合临床，随访。\r\n\r\n,左侧第5、6、7、8肋骨陈旧性骨折，请结合临床，随访。
3401,左肺下野见大片状高密度影，呈外高内低状；左侧膈面欠清晰；左侧肋膈角欠清晰；左侧部分肋骨形态欠规整。,左侧多发肋骨骨折12天复查\r\n左侧胸腔积液（少量）,左侧多发肋骨骨折12天复查；左侧胸腔积液；少量。
3402,主动脉增宽；左侧第六肋骨骨质不连续。,1 左侧第六肋骨疑似骨折，建议结合CT三维进一步明确。\r\n2 主动脉增宽，请结合临床。\r\n,1左侧第六肋骨疑似骨折，建议结合ct三维进一步明确；2主动脉增宽，请结合临床。
3403,左肺见结节状高密度影；左侧第九、十肋骨形态欠规整。,1、左肺散在异常密度影，请结合临床，必要时CT进一步检查。\r\n2、左侧第9、10肋骨形态欠佳。,左肺散在异常密度影，请结合临床，必要时ct进一步检查；左侧第9、10肋骨形态欠佳。
3404,右肺上野见大片状高密度影，呈肺段分布；左侧第八、九肋骨形态欠规整。,左侧第8-9肋骨骨折，右上肺纤增灶（必要时建议CT检查）。\r\n,左侧第8、9肋骨骨折，右上肺纤增灶；必要时建议ct检查。
3405,两肺肺纹理增多、紊乱、模糊；左肺下野见斑片状高密度影；右侧部分肋骨形态欠规整。,左肺感染，请结合临床；\r\n 右侧数个肋骨骨折愈合后改变。\r\n,左肺感染，请结合临床；右侧数个肋骨骨折愈合后改变。


In [270]:
df.to_csv('pure_rib_data.csv', index=False)

In [18]:
test.head()

,clear_finding,frequency,impression
0,中年。男。两肺肺纹理增多、增粗。,2718,"{'心肺未见明显异常。': 379, '两肺未见明显实质性病变。': 311, '两肺纹理增..."
1,中年。女。两肺肺纹理增多、增粗。,2589,"{'心肺未见明显异常。': 275, '两肺未见明显实质性病变。': 216, '请结合临床..."
2,青年。男。两肺肺纹理增多、增粗。,1874,"{'心肺未见明显异常。': 370, '两肺未见明显实质性病变。': 320, '两肺内未见..."
3,青年。女。两肺肺纹理增多、增粗。,1512,"{'心肺未见明显异常。': 319, '两肺内未见明显实质性病变，请结合临床。': 257,..."
4,老年。男。两肺肺纹理增粗、紊乱、模糊。,1074,"{'胸部所示符合慢性支气管炎改变。': 487, '胸部所示符合支气管炎改变。': 81, ..."


# 随机生成印象

In [8]:
from tqdm import tqdm
import random
import pandas as pd 
import os

source = pd.read_csv('output_data_8_12.csv')
out_path = './output_8_12/'

temp = []

for index, row in tqdm(source.iterrows()):
    
    temp.append(random.choice(list(eval((row['impression'])).keys())))
            

source['origin_impression'] = temp
source = source[['clear_finding', 'origin_impression', 'generate_impression', 'impression']]

if not os.path.exists(out_path):
    os.makedirs(out_path)
source.to_csv(out_path + 'f22_with_age.csv', index = None)


32483it [00:01, 24147.30it/s]
